In [5]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

df_train = pd.read_csv("E:\\leonhardt\\mayi\\data\\train.csv")

In [6]:
df_train=df_train.loc[df_train['label']!=-1]
df_train = df_train.fillna(0)
train_set = df_train[df_train.loc[:, "date"]<=20171005]
eva_set = df_train[df_train.loc[:, "date"]>20171005]
train_label = train_set["label"]
eva_label = eva_set["label"]
train_set.drop(["id","date", "label"], axis=1, inplace=True)
eva_set.drop(["id","date", "label"], axis=1, inplace=True)

D:\anaconda\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [25]:
from sklearn.metrics import roc_curve
def atec_metric(preds, labels):
    fpr,tpr,threshhold=roc_curve(labels, preds)
    return 'atec_metric', 0.4*tpr[fpr<=0.001][-1] + 0.3*tpr[fpr<=0.005][-1] + 0.3*tpr[fpr<=0.01][-1], True

In [13]:
from sklearn.ensemble import ExtraTreesClassifier
# for num in range(10, 800, 50):
ext = ExtraTreesClassifier(n_estimators=660, max_depth=5)
ext.fit(train_set, train_label)
eva_prob = ext.predict_proba(eva_set)
eva_score = atec_metric(eva_prob[:, 1], eva_label)
print("{0} is the score of {1}".format(660, eva_score))

660 is the score of ('atec_metric', 0.3389196895295422, True)


In [14]:
df_test = pd.read_csv("E:\\leonhardt\\mayi\\data\\test_a.csv")
df_test.drop(["date","id"],axis=1, inplace=True)
df_test = df_test.fillna(0)

In [15]:
y_pred_ext = ext.predict_proba(df_test)

In [18]:
y_pred_ext.min()

0.003924932985563187

In [19]:
np.savetxt(os.getcwd()+"/prediction/ext_half_fil0_660.txt", y_pred_ext, delimiter=',')

In [21]:
from sklearn.metrics import roc_auc_score,roc_curve
def atec_metric(preds, train_data):
    labels = train_data.get_label()
    fpr,tpr,threshhold=roc_curve(labels, preds)
    return 'atec_metric', 0.4*tpr[fpr<=0.001][-1] + 0.3*tpr[fpr<=0.005][-1] + 0.3*tpr[fpr<=0.01][-1], True

In [20]:
from catboost import CatBoostClassifier

In [29]:
# df_eval = pd.Series(index=range(10, 380, 30))
# for num in range(10, 800, 50):
cat = CatBoostClassifier(iterations=260, depth=8, learning_rate=0.05, logging_level='Verbose', use_best_model=True,
                         eval_metric="AUC")
cat.fit(train_set, train_label, eval_set=[(eva_set, eva_label)], verbose=100)
# eva_prob = cat.predict_proba(eva_set)
# eva_score = atec_metric(eva_prob[:, 1], eva_label)
# print("{0} score is {1}".format(num, eva_score))
# #     df_eval.loc[num, :] = eva_score

0:	learn: 0.9302011	test: 0.9249902	best: 0.9249902 (0)	total: 708ms	remaining: 3m 3s
100:	learn: 0.9816972	test: 0.9726016	best: 0.9726016 (100)	total: 1m 10s	remaining: 1m 50s
200:	learn: 0.9868201	test: 0.9771307	best: 0.9771405 (198)	total: 2m 21s	remaining: 41.5s

bestTest = 0.978507484
bestIteration = 258

Shrink model to first 259 iterations.


In [30]:
y_pred3 = cat.predict_proba(df_test)

In [33]:
np.savetxt(os.getcwd()+"/prediction/cat_half_fil0_260.txt", y_pred3[:,1], delimiter=',')

In [34]:
eva_prob = cat.predict_proba(eva_set)
eva_score = atec_metric(eva_prob[:, 1], eva_label)

In [35]:
eva_score

('atec_metric', 0.49809916046253755, True)